# Detecting Whether a Shot Has Happened

I found out that it is very costly to run detectron keypoint detector and instance predictor on every single frame of a video.
Even with GPU acceleration, processing one second of a video takes around (24 frames) x 1 second = 24 seconds

Thus, it is important to identify the parts of the video where a shot is happening first using a non-DeepLearning method.

This notebook uses non-compute intensive image-processing techniques to locate the basketball and determine when a shot is happening.

Inspired by: https://www.youtube.com/watch?v=QKVpIo5sfGA

In [231]:
import skvideo.io
import skvideo.datasets

from collections import deque

import cv2
import imutils
import numpy as np
import pandas as pd
import math

pd.options.display.max_rows = 1000

# Test a Function on an Image

In [13]:
image = cv2.imread("../data/shooting_vids/blah.png")
output = image.copy()
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# output = locate_bball(gray, output)
# output = get_connected_components(gray, output)

In [22]:
cv2.imshow("test result", np.hstack([output]))
cv2.waitKey(0)

113

# Frame Processing Functions

(some functions are unused in the pipeline. Just there because I experimented different image processing methodologies)

In [2]:
def get_moving_foreground(prev_frame, frame, next_frame):
    """
        Use Three Frame Difference Approach to get moving Foreground Object
        in one frame
        Reference this blog post:
            https://sam-low.com/opencv/frame-differencing.html
    """
    # get differences between frames to detect motion
    diff1 = cv2.absdiff(prev_frame, frame)
    diff2 = cv2.absdiff(frame, next_frame)
    
    # increase contrast between foreground and background by thresholding
    threshold_value = 10
    set_to_value = 255
    _, diff1 = cv2.threshold(diff1, threshold_value, set_to_value, cv2.THRESH_BINARY)
    _, diff2 = cv2.threshold(diff2, threshold_value, set_to_value, cv2.THRESH_BINARY)
    
    # find the overlap between the difference frames to get moving object
    overlap = cv2.bitwise_and(diff1, diff2)
    
    # use median filtering to fill some holes of the moving foreground
    overlap = cv2.medianBlur(overlap,5)
    
    return overlap

In [3]:
def frame_subtraction(bg_frame, cur_frame):
    
    """
        This process of subtracting current frame from bg reference frame
        DOES NOT work well.
        This is because if the camera tilts or shifts slightly the difference
        will be huge. If the leaves of the trees rustle in the background, then
        the difference will be picked up too.
    """
    
    print(type(bg_frame))
    print(type(cur_frame))
    
    diff = cv2.absdiff(bg_frame, cur_frame)
    
    threshold_value = 10
    set_to_value = 255
    _, diff = cv2.threshold(diff, threshold_value, set_to_value, cv2.THRESH_BINARY)
    
    # use median filtering to fill some holes of the moving foreground
    diff = cv2.medianBlur(diff,5)
    
    return diff

In [198]:
def preprocess_vid_frame(frame):
    """
        Do some preprocessing work on a frame of a video to 
        make it ready for image processing techniques.
        1) Convert frame to grayscale
        2) Apply gaussian blur to get rid of noise
    """
    # convert frame to grayscale
    new_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # gaussian blur with 5x5 kernel
    new_frame = cv2.GaussianBlur(new_frame,(5,5), 0)
    
    return new_frame

In [5]:
def stack_frames(frames):
    """
        Use bitwise OR to stack frames together
    """
    res = cv2.bitwise_or(frames[0], frames[1])
    
    for i in range(2, len(frames)):
        res = cv2.bitwise_or(res, frames[i])
    
    return res

In [6]:
def shot_parabola(frame):
    """
        Determine if a shot parabola exists in frame
    """
    
    frame = cv2.Canny(frame,threshold1 = 70, threshold2 = 120)
#     cv2.imshow('canny parabola', frame)

In [7]:
def locate_bball(frame, output):
    """
        Use Hough Circle method
        to locate basketball in frame
        
        Reference: https://www.pyimagesearch.com/2014/07/21/
                   detecting-circles-images-using-opencv-hough-circles/
        
        output frame is the frame you want to draw the
        identifier circle on
    """
    
    # canny edge test
    frame = cv2.Canny(frame,threshold1 = 70, threshold2 = 120)
#     cv2.imshow('canny edge', frame)
    
    
    circles = cv2.HoughCircles(frame, cv2.HOUGH_GRADIENT, dp = 2, minDist = 400,
                               param1 = 5, param2 = 65,
                               minRadius = 0, maxRadius = 60)
    
#     circles = cv2.HoughCircles(frame, cv2.HOUGH_GRADIENT, dp = 5, minDist = 20,
#                                param1 = 50, param2 = 100,
#                                minRadius = 0, maxRadius = 300)

    if circles is not None:
        circles = np.round(circles[0, :]).astype("int")
        
        for (x,y,r) in circles:
            # draw the outer circle
            cv2.circle(output, (x, y), r, (0, 255, 0), 4)
            # draw the center of the circle
            cv2.circle(output, (x, y), 2, (0, 255, 0), 3)
    
    return output

In [8]:
def get_connected_components(frame, output):
    """
        Get connected pixel blobs
        Reference: https://stackoverflow.com/questions/35854197/
                   how-to-use-opencvs-connected-components-with-stats-in-python
                   
        Input frame is a frame that has been thresholded
        Output frame is frame to draw connected component results on
    """
    
    connectivity = 4
    cc = cv2.connectedComponentsWithStats(frame, connectivity, cv2.CV_32S)
    
    # The first cell is the number of labels
    num_labels = cc[0]
    # The second cell is the label matrix
    labels = cc[1]
    # The third cell is the stat matrix
    stats = cc[2]
    # The fourth cell is the centroid matrix
    centroids = cc[3]
    
    print(num_labels)
    print(labels)
    print(stats)
    print(centroids)
    for (x,y) in centroids:
        # draw the center of the circle
        x, y = int(x), int(y)
        cv2.circle(output, (x, y), 2, (0, 255, 0), 3)
        
    return output

In [42]:
def get_blobs(frame, output):
    """
        Get all the blobs (connected components of white pixels)
        in a thresholded black and white image
        
        Reference: https://www.pyimagesearch.com/2015/05/25/
                   basic-motion-detection-and-tracking-with-python-and-opencv/
                   
        Blobs info is a list of dictionaries containing coordinates
        and size of blobs
    """
    
    cnts = cv2.findContours(frame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    
    blobs_info = {}
    
    # loop over the contours
    for c in cnts:
        # if the contour is too small, ignore it
        if cv2.contourArea(c) < 100:
            continue
            
        # find the bounding box for the contour
        (x, y, w, h) = cv2.boundingRect(c)
        cv2.rectangle(output, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
        blobs_info['x'], blobs_info['y'] = x, y
        blobs_info['width'], blobs_info['height'] = w, h
    
    return output, blobs_info

# Run Video Processing - Find all blobs

In [295]:
def find_blobs(video):
    # record the last 3 frames of video for foreground detection
    frames = deque()
    frames.append(preprocess_vid_frame(video.read()[1]))
    frames.append(preprocess_vid_frame(video.read()[1]))
    frames.append(preprocess_vid_frame(video.read()[1]))

    # record the last 50 frames of foreground detection for frame stacking
    frames_foreground = deque()


    # record blobs information
    blobs_info = []

    frame_count = 0
    while(video.isOpened()):
        frame_count += 1

        # current frame of consideration is the middle frame in frames deque
        cur_frame = frames[1]


        # find moving foreground
        foreground = get_moving_foreground(frames[0], frames[1], frames[2])
        cv2.imshow('Foreground', foreground)


        # find blobs and record blobs information
        blobs_frame = cv2.cvtColor(cur_frame, cv2.COLOR_GRAY2RGB)
        blobs_frame, b_info = get_blobs(foreground, blobs_frame)
        b_info['frame_idx'] = frame_count
        blobs_info.append(b_info)
        cv2.imshow('Detected Blobs',blobs_frame)


        # stack last 10 foreground frames together
    #     frames_foreground.append(foreground)
    #     if (len(frames_foreground) > 50):
    #         frames_foreground.popleft()
    #         stacked_foregrounds = stack_frames(frames_foreground)
    #         cv2.imshow('Foregrounds stacked', stacked_foregrounds)
    #         shot_parabola(stacked_foregrounds)


        if cv2.waitKey(15) & 0xFF == ord('q'):
            break

        # get next frame
        frames.popleft()
        ret, next_frame = video.read()
        if ret == False:
            break
        frames.append(preprocess_vid_frame(next_frame))
    
    width  = video.get(cv2.CAP_PROP_FRAME_WIDTH)
    height = video.get(cv2.CAP_PROP_FRAME_HEIGHT)
    
    video.release()
    cv2.destroyAllWindows()

    blobs_df = pd.DataFrame(blobs_info)
    
    
    
    return blobs_df, (width, height)

In [296]:
video = cv2.VideoCapture("../data/shooting_vids/shooting6.mp4")

blobs_df, frame_dim = find_blobs(video)

In [268]:
# blobs_df.iloc[0:1000]

# Function to Find Bball Blobs

In [291]:
def find_bball_blobs(blobs_info_df, frame_size):
    """
        Find all the blobs that are the basketball and label them
    """
    
    blobs_df = blobs_info_df.copy(deep = True)
    
    # convert y axis so that it goes from bottom to top
    blobs_df['y'] = frame_size[1] - blobs_df['y']
    
    # create column in pandas dataframe for labeling blob as bball
    blobs_df['is_bball'] = False
    
    # hyperparameters to determine what blobs count as a basketball
    BLOB_DIST = 100
    BLOB_SIZE_TOL = 0.3
    
    def get_dist(row, cur_x, cur_y):
        return ((row['x'] - cur_x)**2 + (row['y'] - cur_y)**2)**(1/2)

    def similar_shape(row, cur_w, cur_h):
        new_w, new_h = row['width'], row['height']
        tol_w, tol_h = cur_w * BLOB_SIZE_TOL, cur_h * BLOB_SIZE_TOL

        tol_w_lower, tol_w_upper = cur_w - tol_w, cur_w + tol_w
        tol_h_lower, tol_h_upper = cur_h - tol_h, cur_h + tol_h

        return (new_w >= tol_w_lower and new_w <= tol_w_upper and
                new_h >= tol_h_lower and new_h <= tol_h_upper)

    def find_ball_blob(blob_idx, time_forward):
        """
            Find the ball blob that is closest to the current ball blob
            that is similar in size timewise. Return None is no ball 
            blob can be found
        """
        cur_blob = blobs_df.loc[blob_idx]
        cur_x, cur_y = cur_blob['x'], cur_blob['y']
        cur_w, cur_h = cur_blob['width'], cur_blob['height']
        
        if math.isnan(cur_x):
            return None
        
        # get all the blobs one timestep forward or one timestep backwards
        cur_frame_idx = blobs_df.loc[blob_idx]['frame_idx']
        frame_idx = (cur_frame_idx + 1) if time_forward else (cur_frame_idx - 1)
        blobs = blobs_df[blobs_df['frame_idx'] == frame_idx].copy(deep = True)

        # find the distance of all blobs to current blob
        blobs['distance'] = blobs.apply(get_dist, cur_x = cur_x, cur_y = cur_y, axis = 1)

        # determine of blobs are similar shape to current blob
        blobs['similar_shape'] = blobs.apply(similar_shape, cur_w = cur_w, cur_h = cur_h, axis = 1)

        valid_blobs = blobs[(blobs['distance'] < BLOB_DIST) & (blobs['similar_shape'])]

        # only return blob index if there is one valid blob
        # multiple blobs mean algorithm cannot discern basketball between multiple blobs
#         print(valid_blobs)
        if valid_blobs.shape[0] == 1:
            print("RETURNED STH USEFUL")
            return valid_blobs.index[0]
        else:
            return None
    
    
    def get_idx_ranges(indices):
        """
            Get the start and end value of continuous indices
            [1,2,3,4,7,8,9,14,15,16] --> [(1,4), (7,9), (14,16)]
            
            Expects a sorted list of indices
        """
        start, end = indices[0], 0
        cur = start + 1
        index_ranges = []
        for i in indices[1:]:
            if i != cur:
                end = cur - 1
                index_ranges.append((start, end))
                start = i
                cur = i + 1
            else:
                cur += 1
            if i == a[-1]:
                end = i
                index_ranges.append((start, end))
        return index_ranges
    
    
    
    # any blob above 95% of max blob height is considered bball
    # the 95% is used to account for varying shot arcs (ball reaches different heights)
    ball_max_height = blobs_df['y'].max()
    ball_effective_height = ball_max_height * 0.80
    blobs_df.loc[blobs_df['y'] >= ball_effective_height, 'is_bball'] = True
    
    
    # get the indices of is_bball == True already
    ball_idxs = blobs_df.index[blobs_df['is_bball']].tolist()
    ball_idx_ranges = get_idx_ranges(ball_idxs)
    
    print("BALL INDEX RANGES")
    print(ball_idx_ranges)
    print()
    for i, (ball_start, ball_end) in enumerate(ball_idx_ranges):
        
#         if i == 2:
#             break
        
        print((ball_start, ball_end))
        # trace the balls until they can't be found backwards/forwards in time
        ball_idx = ball_start
        while(True):
            # get previous ball index
            ball_idx = find_ball_blob(ball_idx, time_forward = False)
            # if no ball is found, stop looking
            if ball_idx is None:
                print("no ball found 1")
                break
            # label ball
            blobs_df.loc[ball_idx, "is_bball"] = True
        
            
        # trace the balls until they can't be found forwards in time
        ball_idx = ball_end
        while(True):
            # get previous ball index
            ball_idx = find_ball_blob(ball_idx, time_forward = True)
            # if no ball is found, stop looking
            if ball_idx is None:
                print('no ball found 2')
                break
            # label ball
            blobs_df.loc[ball_idx, "is_bball"] = True
        
    
    # convert y axis back so that it can be plotted by opencv
    blobs_df['y'] = frame_dim[1] - blobs_df['y']
    
    return blobs_df

In [292]:
res_df = find_bball_blobs(blobs_df, frame_dim)
res_df.head()

BALL INDEX RANGES
[(183, 187), (424, 437), (700, 707), (709, 714), (716, 719), (919, 926), (928, 939), (1099, 1104), (1106, 1111), (1113, 1117), (1276, 1282), (1284, 1285), (1287, 1295), (1487, 1506), (1664, 1680), (1878, 1884), (1888, 1889), (1892, 1899), (2175, 2191), (2193, 2193), (2294, 2306), (2308, 2311), (2495, 2512), (2611, 2622), (2928, 2942), (3151, 3185)]

(183, 187)
no ball found 1
no ball found 2
(424, 437)
RETURNED STH USEFUL
RETURNED STH USEFUL
RETURNED STH USEFUL
RETURNED STH USEFUL
RETURNED STH USEFUL
RETURNED STH USEFUL
RETURNED STH USEFUL
RETURNED STH USEFUL
RETURNED STH USEFUL
no ball found 1
no ball found 2
(700, 707)
RETURNED STH USEFUL
RETURNED STH USEFUL
RETURNED STH USEFUL
RETURNED STH USEFUL
RETURNED STH USEFUL
RETURNED STH USEFUL
RETURNED STH USEFUL
RETURNED STH USEFUL
RETURNED STH USEFUL
no ball found 1
no ball found 2
(709, 714)
no ball found 1
no ball found 2
(716, 719)
no ball found 1
no ball found 2
(919, 926)
RETURNED STH USEFUL
RETURNED STH USEFUL
RETU

,frame_idx,x,y,width,height,is_bball
0,1,NaN,NaN,NaN,NaN,False
1,2,NaN,NaN,NaN,NaN,False
2,3,NaN,NaN,NaN,NaN,False
3,4,NaN,NaN,NaN,NaN,False
4,5,NaN,NaN,NaN,NaN,False


# Function to plot basketball

In [287]:
def plot_bball(video, ball_df):
    
    # record the last 3 frames of video for foreground detection
    frames = deque()
    frames.append(preprocess_vid_frame(video.read()[1]))
    frames.append(preprocess_vid_frame(video.read()[1]))
    frames.append(preprocess_vid_frame(video.read()[1]))


    frame_count = 0
    while(video.isOpened()):
        frame_count += 1

        # current frame of consideration is the middle frame in frames deque
        cur_frame = frames[1]
        

        # find moving foreground
        foreground = get_moving_foreground(frames[0], frames[1], frames[2])
        cv2.imshow('Foreground', foreground)


        # label basketball in each frame
        bballs_labeled = cv2.cvtColor(cur_frame.copy(), cv2.COLOR_GRAY2BGR)
        blobs = ball_df[(ball_df['frame_idx'] == frame_count) &
                        (ball_df['is_bball'])]
        for _, blob in blobs.iterrows():
            x, y, w, h = int(blob['x']), int(blob['y']), int(blob['width']), int(blob['height'])
            cv2.rectangle(bballs_labeled, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(bballs_labeled, "Basketball!", (x, y),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        cv2.imshow('Bballs Labeled', bballs_labeled)


        if cv2.waitKey(20) & 0xFF == ord('q'):
            break

        # get next frame
        frames.popleft()
        ret, next_frame = video.read()
        if ret == False:
            break
        frames.append(preprocess_vid_frame(next_frame))

    video.release()
    cv2.destroyAllWindows()

In [289]:
video = cv2.VideoCapture("../data/shooting_vids/shooting6.mp4")

plot_bball(video, res_df)

In [290]:
import cv2
cap = cv2.VideoCapture("../data/shooting_vids/shooting6.mp4")
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

def onChange(trackbarValue):
    cap.set(cv2.CAP_PROP_POS_FRAMES,trackbarValue)
    err,img = cap.read()
    cv2.imshow("mywindow", img)
    pass

cv2.namedWindow('mywindow')
cv2.createTrackbar( 'start', 'mywindow', 0, length, onChange )
cv2.createTrackbar( 'end'  , 'mywindow', 100, length, onChange )

onChange(0)
cv2.waitKey()

start = cv2.getTrackbarPos('start','mywindow')
end   = cv2.getTrackbarPos('end','mywindow')
if start >= end:
    raise Exception("start must be less than end")

cap.set(cv2.CAP_PROP_POS_FRAMES,start)
while cap.isOpened():
    err,img = cap.read()
    if cap.get(cv2.CAP_PROP_POS_FRAMES) >= end:
        break
    cv2.imshow("mywindow", img)
    k = cv2.waitKey(10) & 0xff
    if k==27:
        break

error: OpenCV(4.3.0) /Users/travis/build/skvark/opencv-python/opencv/modules/highgui/src/window.cpp:376: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'imshow'


error: OpenCV(4.3.0) /Users/travis/build/skvark/opencv-python/opencv/modules/highgui/src/window.cpp:376: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'imshow'


error: OpenCV(4.3.0) /Users/travis/build/skvark/opencv-python/opencv/modules/highgui/src/window.cpp:376: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'imshow'


error: OpenCV(4.3.0) /Users/travis/build/skvark/opencv-python/opencv/modules/highgui/src/window.cpp:376: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'imshow'


error: OpenCV(4.3.0) /Users/travis/build/skvark/opencv-python/opencv/modules/highgui/src/window.cpp:376: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'imshow'


KeyboardInterrupt: 